In [4]:
import numpy as np

# Definizione degli stati
states = ["Rettilineo", "Curva", "Fuori strada", "Macchina davanti in rettilineo", "Macchina davanti in curva", "Incidente"]
num_states = len(states)

# Definizione delle azioni (combinazione di sterzata e velocità)
actions = [(30, 50), (30, 90), (30, 130), (60, 50), (60, 90), (60, 130), (90, 50), (90, 90), (90, 130)]
num_actions = len(actions)

# Matrice di transizione P(s' | s, a) -> dimensioni (num_states, num_states, num_actions)
P = np.zeros((num_states, num_states, num_actions))

# Riempimento della matrice di transizione
for a in range(num_actions):
    sterzata, velocita = actions[a]
    
    for s in range(num_states):
        if states[s] == "Rettilineo":
            if sterzata == 30 and velocita == 50:
                P[s, 0, a] = 0.8  # Rimane in rettilineo
                P[s, 1, a] = 0.15 # Va in curva
                P[s, 2, a] = 0.05 # Fuori strada
            elif sterzata == 90 or velocita == 130:
                P[s, 0, a] = 0.2  # Rimane in rettilineo
                P[s, 1, a] = 0.4  # Va in curva
                P[s, 2, a] = 0.4  # Fuori strada
            else:
                P[s, 0, a] = 0.5  # Rimane in rettilineo
                P[s, 1, a] = 0.3  # Va in curva
                P[s, 2, a] = 0.2  # Fuori strada
        elif states[s] == "Curva":
            if sterzata == 30 or sterzata == 60:
                P[s, 1, a] = 0.7  # Rimane in curva
                P[s, 0, a] = 0.1  # Torna in rettilineo
                P[s, 2, a] = 0.2  # Fuori strada
            else:
                P[s, 2, a] = 0.6  # Fuori strada
                P[s, 1, a] = 0.3  # Rimane in curva
                P[s, 5, a] = 0.1  # Incidente
        elif states[s] == "Macchina davanti in rettilineo":
            if velocita == 130:
                P[s, 5, a] = 0.6  # Incidente
                P[s, 0, a] = 0.2  # Rimane in rettilineo
                P[s, 3, a] = 0.2  # Ancora macchina davanti
            else:
                P[s, 3, a] = 0.7  # Rimane in stato macchina davanti
                P[s, 0, a] = 0.2  # Rimane in rettilineo
                P[s, 5, a] = 0.1  # Incidente
        elif states[s] == "Macchina davanti in curva":
            if velocita == 130:
                P[s, 5, a] = 0.5  # Incidente
                P[s, 1, a] = 0.3  # Rimane in curva
                P[s, 4, a] = 0.2  # Ancora macchina davanti in curva
            else:
                P[s, 4, a] = 0.6  # Rimane nello stato di macchina davanti in curva
                P[s, 1, a] = 0.3  # Rimane in curva
                P[s, 5, a] = 0.1  # Incidente
        elif states[s] == "Fuori strada":
            P[s, 2, a] = 0.8  # Rimane fuori strada
            P[s, 5, a] = 0.2  # Incidente
        elif states[s] == "Incidente":
            P[s, 5, a] = 1.0  # Rimane nello stato di incidente (assorbente)

# Funzione per verificare che ogni riga della matrice di transizione sia una distribuzione di probabilità valida
def validate_transition_matrix(P):
    for a in range(num_actions):
        for s in range(num_states):
            row_sum = np.sum(P[s, :, a])
            if not np.isclose(row_sum, 1.0):
                print(f"Errore: la somma delle probabilità per lo stato '{states[s]}' e l'azione {actions[a]} è {row_sum:.2f}, non 1.0")
            else:
                print(f"Stato '{states[s]}', azione {actions[a]}: somma delle probabilità corretta ({row_sum:.2f})")

# Validazione della matrice di transizione
validate_transition_matrix(P)

# Stampa delle probabilità di transizione per alcune azioni
for a in range(num_actions):
    print(f"\nTransizioni per l'azione {actions[a]}:")
    for s in range(num_states):
        print(f"  Stato attuale: {states[s]}")
        for s_next in range(num_states):
            if P[s, s_next, a] > 0:
                print(f"    -> {states[s_next]}: {P[s, s_next, a]:.2f}")


Stato 'Rettilineo', azione (30, 50): somma delle probabilità corretta (1.00)
Stato 'Curva', azione (30, 50): somma delle probabilità corretta (1.00)
Stato 'Fuori strada', azione (30, 50): somma delle probabilità corretta (1.00)
Stato 'Macchina davanti in rettilineo', azione (30, 50): somma delle probabilità corretta (1.00)
Stato 'Macchina davanti in curva', azione (30, 50): somma delle probabilità corretta (1.00)
Stato 'Incidente', azione (30, 50): somma delle probabilità corretta (1.00)
Stato 'Rettilineo', azione (30, 90): somma delle probabilità corretta (1.00)
Stato 'Curva', azione (30, 90): somma delle probabilità corretta (1.00)
Stato 'Fuori strada', azione (30, 90): somma delle probabilità corretta (1.00)
Stato 'Macchina davanti in rettilineo', azione (30, 90): somma delle probabilità corretta (1.00)
Stato 'Macchina davanti in curva', azione (30, 90): somma delle probabilità corretta (1.00)
Stato 'Incidente', azione (30, 90): somma delle probabilità corretta (1.00)
Stato 'Rettili

In [3]:
# Esempio di correzione per lo stato 'Macchina davanti in curva'
transizioni = {
    'Rettilineo': {'Rettilineo': 0.80, 'Curva': 0.15, 'Fuori strada': 0.05},
    'Curva': {'Rettilineo': 0.10, 'Curva': 0.70, 'Fuori strada': 0.20},
    'Fuori strada': {'Fuori strada': 0.80, 'Incidente': 0.20},
    'Macchina davanti in rettilineo': {'Rettilineo': 0.20, 'Macchina davanti in rettilineo': 0.70, 'Incidente': 0.10},
    'Macchina davanti in curva': {'Curva': 0.60, 'Macchina davanti in curva': 0.30, 'Incidente': 0.10},  # Aggiunto
    'Incidente': {'Incidente': 1.00}
}

# Funzione per controllare le somme delle probabilità
for stato, azioni in transizioni.items():
    somma_prob = sum(azioni.values())
    if abs(somma_prob - 1.0) > 1e-6:
        print(f"Errore: la somma delle probabilità per lo stato '{stato}' è {somma_prob:.2f}, non 1.0")
    else:
        print(f"Stato '{stato}': somma delle probabilità corretta (1.00)")


Stato 'Rettilineo': somma delle probabilità corretta (1.00)
Stato 'Curva': somma delle probabilità corretta (1.00)
Stato 'Fuori strada': somma delle probabilità corretta (1.00)
Stato 'Macchina davanti in rettilineo': somma delle probabilità corretta (1.00)
Stato 'Macchina davanti in curva': somma delle probabilità corretta (1.00)
Stato 'Incidente': somma delle probabilità corretta (1.00)


In [18]:
import numpy as np

# Variabile aleatoria per il rumore nella sterzata (distribuzione normale)
def add_noise(sterzata, mean=0, std=5):
    """
    Aggiunge rumore alla sterzata utilizzando una distribuzione normale.
    mean: media del rumore (default 0)
    std: deviazione standard del rumore (default 5)
    """
    noise = np.random.normal(mean, std)
    sterzata_noisy = sterzata + noise
    return np.clip(sterzata_noisy, 0, 90)  # Limita la sterzata tra 0 e 90 gradi

# Esempio di utilizzo
sterzata_originale = 30
sterzata_con_rumore = add_noise(sterzata_originale)
print(f"Sterzata originale: {sterzata_originale}, Sterzata con rumore: {sterzata_con_rumore:.2f}")


Sterzata originale: 30, Sterzata con rumore: 34.43


In [24]:
import numpy as np
from scipy.stats import norm

# Funzione per calcolare la sterzata massima sicura in funzione della velocità
def compute_delta_max(v, a_y_max, L):
    """
    Calcola la sterzata massima sicura delta_max(v) in gradi.
    
    Parametri:
      v       : velocità del veicolo (m/s)
      a_y_max : accelerazione laterale massima sicura (m/s^2)
      L       : lunghezza del veicolo (m)
      
    Ritorna:
      delta_max in gradi.
    """
    # Calcola delta_max in radianti:
    delta_max_rad = np.arctan((a_y_max * L) / (v**2))
    # Converte in gradi
    return np.degrees(delta_max_rad)

# Funzione per calcolare la probabilità di uscire dalla carreggiata
def offroad_probability(delta_cmd, v, sigma, a_y_max, L):
    """
    Calcola la probabilità che, a causa del rumore, il veicolo esca dalla carreggiata.
    
    Parametri:
      delta_cmd : sterzata comandata (gradi)
      v         : velocità del veicolo (m/s)
      sigma     : deviazione standard del rumore nella sterzata (gradi)
      a_y_max   : accelerazione laterale massima sicura (m/s^2)
      L         : lunghezza del veicolo (m)
    
    La sterzata effettiva è data da:
         delta_eff = delta_cmd + epsilon, con epsilon ~ N(0, sigma^2)
    
    Il veicolo esce dalla carreggiata se:
         |delta_eff| > delta_max(v)
    
    La probabilità è quindi:
         P(off-road) = 1 - Phi((delta_max(v) - delta_cmd) / sigma) + Phi((-delta_max(v) - delta_cmd) / sigma)
    """
    # Calcola la soglia di sterzata sicura in gradi
    delta_max = compute_delta_max(v, a_y_max, L)
    
    # Calcola i valori z per la gaussiana standard
    z1 = (delta_max - delta_cmd) / sigma
    z2 = (-delta_max - delta_cmd) / sigma
    
    # Calcola la probabilità utilizzando la CDF della normale standard
    p_offroad = 1 - norm.cdf(z1) + norm.cdf(z2)
    return p_offroad

# Esempio di utilizzo:
if __name__ == "__main__":
    # Parametri di esempio
    delta_cmd = 15    # sterzata comandata in gradi
    v = 20            # velocità in m/s (circa 72 km/h)
    sigma = 5         # deviazione standard del rumore in gradi
    a_y_max = 50       # accelerazione laterale massima sicura in m/s^2
    L = 2.5           # lunghezza del veicolo in m

    # Calcola la sterzata massima sicura a questa velocità
    delta_max = compute_delta_max(v, a_y_max, L)
    print(f"Sterzata massima sicura a {v} m/s: {delta_max:.2f} gradi")

    # Calcola la probabilità di uscita dalla carreggiata
    p = offroad_probability(delta_cmd, v, sigma, a_y_max, L)
    print(f"Probabilità di uscire dalla carreggiata: {p:.4f}")


Sterzata massima sicura a 20 m/s: 17.35 gradi
Probabilità di uscire dalla carreggiata: 0.3189


---> CDF

In [1]:
from scipy.stats import norm
1 - norm.cdf(0.40012)

0.3445340670335465

In [3]:
1 - norm.cdf(1.95041)

0.025563635457655387

In [38]:
norm.cdf(-1.38356)

0.08324661171013015

In [39]:
(1 - norm.cdf(-1.01644)) + norm.cdf(-1.38356)

0.9285366588084327

In [36]:
import numpy as np
from scipy.stats import norm

def offroad_probability(y_t, theta_t, v_t, dt, L_max):
    """
    Calcola la probabilità che, in un intervallo dt, il veicolo esca dalla carreggiata,
    cioè che y_(t+1) > L_max.
    
    Il modello è:
        y_(t+1) = y_t + v_t * sin(theta_t + δ_t) * dt,
    dove δ_t ~ N(0, v_t) (ovvero con deviazione standard sqrt(v_t))
    
    Parametri:
      y_t     : spostamento laterale attuale (m)
      theta_t : sterzata comandata (radianti) [valore deterministico]
      v_t     : velocità del veicolo (m/s)
      dt      : intervallo di tempo (s)
      L_max   : distanza laterale massima dal centro della carreggiata (m)
    
    Ritorna:
      p_offroad: probabilità che y_(t+1) > L_max
    """
    # Calcola s = (L_max - y_t) / (v_t * dt)
    s = (L_max - y_t) / (v_t * dt)
    # Limitiamo s tra -1 e 1 per poter usare arcsin in modo corretto
    s_clipped = np.clip(s, -1, 1)
    
    # Calcola la soglia angolare delta_threshold = arcsin(s_clipped)
    delta_threshold = np.arcsin(s_clipped)
    
    # Il veicolo esce se: theta_t + δ_t > delta_threshold, cioè se δ_t > delta_threshold - theta_t.
    # Poiché δ_t ~ N(0, v_t), la deviazione standard è sqrt(v_t)
    sigma = np.sqrt(v_t)
    
    # Calcola il valore z per la distribuzione normale standard
    z = (delta_threshold - theta_t) / sigma
    
    # La probabilità che δ_t superi questo valore è:
    p_offroad = 1 - norm.cdf(z)
    
    return p_offroad

# Esempio di utilizzo:
if __name__ == "__main__":
    # Parametri di esempio:
    y_t = 0                   # spostamento laterale attuale (m)
    theta_t = np.radians(30)      # sterzata comandata (5° in radianti)
    v_t = 25.0                   # velocità in m/s
    dt = 0.1                     # intervallo di tempo (s)
    L_max = 2                 # limite laterale (m)
    
    p = offroad_probability(y_t, theta_t, v_t, dt, L_max)
    print("Probabilità di uscire dalla carreggiata (y_(t+1) > L_max): {:.4f}".format(p))


Probabilità di uscire dalla carreggiata (y_(t+1) > L_max): 0.4134


In [47]:
import numpy as np
from scipy.stats import norm

def offroad_probability(y_t, theta_t, v_t, dt, L_max):
    """
    Calcola la probabilità che, in un intervallo dt, il veicolo esca dalla carreggiata,
    cioè che y_(t+1) > L_max.
    
    Il modello:
        y_(t+1) = y_t + v_t * sin(theta_t + δ_t) * dt,
    con δ_t ~ N(0, v_t)   (deviazione standard: sqrt(v_t)).
    
    L'uscita dalla carreggiata avviene se:
        theta_t + δ_t > δ_threshold, dove δ_threshold = arcsin( (L_max - y_t) / (v_t * dt) ).
    
    Parametri:
      y_t     : posizione laterale attuale (m)
      theta_t : sterzata comandata (radianti) - valore deterministico
      v_t     : velocità del veicolo (m/s)
      dt      : intervallo di tempo (s)
      L_max   : distanza laterale massima dal centro della carreggiata (m)
    
    Ritorna:
      p_offroad: probabilità che y_(t+1) > L_max
    """
    
    # Calcola s e limita il suo valore tra -1 e 1 (per arcsin)
    s = (L_max - y_t) / (v_t * dt)
    s_clipped = np.clip(s, -1, 1)
    
    # Soglia angolare in radianti
    delta_threshold = np.arcsin(s_clipped)
    
    # L'uscita avviene se: θ_t + δ_t > delta_threshold  =>  δ_t > delta_threshold - θ_t.
    # Poiché δ_t ~ N(0, v_t), la deviazione standard è sqrt(v_t)
    sigma = v_t
    
    # Calcola il valore z
    z = (delta_threshold - theta_t- 30) / sigma
    
    # Probabilità che δ_t superi questo valore
    p_offroad = 1 - norm.cdf(z)
    
    return p_offroad

# Esempio: 
# Dati:
#   y_t      = 0 m 
#   theta_t  = 0 gradi  -> 0 radianti
#   v_t      = 90 km/h = 25 m/s (circa)
#   L_max    = 2 m
#   δ_t  soglia: per far sì che delta_threshold sia pari a 30° (0.5236 rad)
#   Risolviamo per dt:  delta_threshold = arcsin( (L_max - y_t)/(v_t*dt) ) = 0.5236 rad 
#         => (L_max)/(v_t*dt) = sin(0.5236) = 0.5  =>  dt = L_max/(v_t*0.5) = 2/(25*0.5) = 0.16 s

if __name__ == "__main__":
    y_t = 0.0                   # posizione laterale attuale (m)
    theta_t = 0.0               # sterzata comandata (radianti)
    v_kmh = 90                  # velocità in km/h
    v_t = v_kmh / 3.6           # conversione in m/s: 90 km/h ≈ 25 m/s
    L_max = 2.0                 # limite laterale (m)
    dt = 1                      # intervallo di tempo scelto (s) in modo che delta_threshold = 30° (0.5236 rad)
    
    # Calcolo della soglia
    s = (L_max - y_t) / (v_t * dt)
    s_clipped = np.clip(s, -1, 1)
    delta_threshold = np.arcsin(s_clipped)
    print("δ_threshold (in gradi): {:.2f}".format(np.degrees(delta_threshold)))
    
    # Calcola la probabilità di uscita dalla carreggiata
    p = offroad_probability(y_t, theta_t, v_t, dt, L_max)
    print("Probabilità di uscire dalla carreggiata: {:.4f}".format(p))
    print("Probabilità di rimanere in carreggiata: {:.4f}".format(1 - p))



δ_threshold (in gradi): 4.59
Probabilità di uscire dalla carreggiata: 0.8843
Probabilità di rimanere in carreggiata: 0.1157
